# [Scrapy Beginners Series Part 1: How To Build Your First Production Scraper](https://scrapeops.io/python-scrapy-playbook/scrapy-beginners-guide/)

# Scrapy 初心者向けシリーズ パート 1: 初めてのプロダクション スクレーパーを構築する方法

- Python Scrapy 5部構成の初心者向けシリーズ
- Part 1: Basic Scrapy Spider
  - Scrapyの基本を確認し、最初のScrapyスパイダーを構築します。
- Part 2：汚れたデータのクリーニングとエッジケースの処理
  - Webデータは厄介で構造化されておらず、多くのエッジケースを持つことができます。
  - Items、Itemloaders、Item Pipelineを使用して、これらのエッジケースに対してスパイダーを堅牢にします。
- Part 3：データの保存
  - データベース、CSVファイル、JSONフォーマット、S3バケットからスクレイピングしたデータを保存する方法はさまざまです。
  - データを保存するさまざまな方法を検討し、その長所、短所、どのような場面で使うかについて説明します。
- Part4：ユーザーエージェントとプロキシ
  - ユーザーエージェントとIPを管理し、ブロックされないようにすることで、スパイダーを本番環境に対応させます。
- Part 5: デプロイメント、スケジューリング、ジョブの実行
  - サーバー上にスパイダーをデプロイし、[ScrapeOps](https://scrapeops.io/)を介してジョブの監視とスケジューリングを行う。

- CSSセレクタとXPath表現の解析
- データ形式（CSV、JSON、XML）およびストレージ（FTP、S3、ローカルファイルシステム）
- 堅牢なエンコーディングサポート
- 同時実行管理
- 自動再試行
- クッキーとセッションの処理
- クロールスパイダーと内蔵のページネーションサポート

## Beginners Scrapy Tutorial

### Step 1 - Setup your Python Environment

#### Test Scrapy Is Installed

```bash
(venv) c01 (😁 :main *) :$ scrapy

Scrapy 2.9.0 - no active project

Usage:
  scrapy <command> [options] [args]

Available commands:
  bench         Run quick benchmark test
  fetch         Fetch a URL using the Scrapy downloader
  genspider     Generate new spider using pre-defined templates
  runspider     Run a self-contained spider (without creating a project)
  settings      Get settings values
  shell         Interactive scraping console
  startproject  Create new project
  version       Print Scrapy version
  view          Open URL in browser, as seen by Scrapy

  [ more ]      More commands available when run from project directory

Use "scrapy <command> -h" to see more info about a command
```

### Step 2 - Setup Our Scrapy Project

```bash
# scrapyプロジェクトの作成
(venv) c01 (😁 :main *) :$ scrapy startproject chocolatescraper

New Scrapy project 'chocolatescraper', using template directory '/Users/takeru/@LEARNING/Python/python-scrapy-playbook-scrapyops/venv/lib/python3.10/site-packages/scrapy/templates/project', created in:
    /Users/takeru/@LEARNING/Python/python-scrapy-playbook-scrapyops/c01/chocolatescraper

You can start your first spider with:
    cd chocolatescraper
    scrapy genspider example example.com
```

```bash
# chocolatescraperディレクトリに移動
cd chocolatescraper
```

```bash
# chocolatescraperディレクトリのtree
(venv) chocolatescraper (😁 :main *) :$ tree

.
├── chocolatescraper
│   ├── __init__.py
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       └── __init__.py
└── scrapy.cfg

3 directories, 7 files
```

- `settings.py`
  - パイプラインやミドルウェアの有効化など、プロジェクトのすべての設定が含まれる場所です。
  - 遅延、同時実行、その他多くのことを変更することができます。
- `items.py`
  - 抽出されたデータのモデルです。
  - Scrapy Itemクラスを継承し、スクレイピングされたデータを含むカスタムモデル（ProductItemのような）を定義することができます
- `pipelines.py`
  - スパイダーが生成したアイテムが渡される場所で、主にテキストのクリーニングやファイル出力やデータベース（CSV、JSON SQLなど）への接続に使用します。
- `middlewares.py`
  - リクエストが行われ、scrapy がレスポンスを処理する方法を変更したいときに便利です。
- `scrapy.cfg`
  - いくつかのデプロイ設定などを変更するための設定ファイルです。

### Step 3- Creating Our Spider

- Spider
  - `start_url` のリストを受け取り、それぞれを `parse` メソッドでスクレイピングする。
- CrawlSpider 
  - 見つけたリンクをたどって、ウェブサイト全体をクロールするように設計されている。
- SitemapSpider
  - サイトマップから URL を抽出するように設計されてる。

```bash
# 新しい汎用スパイダーを作成するには、`genspider`コマンドを実行するだけです：
# scrapy genspider <name_of_spider> <website>
(venv) chocolatescraper (😁 :main *) :$ scrapy genspider chocolatespider chocolate.co.uk

Created spider 'chocolatespider' using template 'basic' in module:
  chocolatescraper.spiders.chocolatespider
```

これで `spiders` フォルダに新しいスパイダーが追加され、次のようになります：

```python
# chocolatespider.py

import scrapy

class ChocolatespiderSpider(scrapy.Spider):
    name = 'chocolatespider'
    allowed_domains = ['chocolate.co.uk']
    start_urls = ['http://chocolate.co.uk/']

    def parse(self, response):
        pass
```

ここでは、`genspider`コマンドが、`Spider`クラスという形で、私たちが使用するためのテンプレートスパイダーを作成したことを確認します。

このスパイダー・クラスには以下のものがあります：

- name
  - スパイダーに名前をつけるクラス属性です。
  - あとで `scrapy crawl <spider_name>` としてスパイダーを実行するときに、この名前を使用します。
- allowed_domains
  - クラス属性で、Scrapyに `chocolate.co.uk` ドメインのページのみをスクレイピングするように指示します。
  - スパイダーが暴走して多くのウェブサイトをスクレイピングするのを防ぐためです。
  - これはオプションです。
- start_urls
  - Scrapyにスクレイピングすべき最初のURLを指示するクラス属性です。
  - これは少しづつ変更していきます。
- parse
  - ターゲットウェブサイトからレスポンスを受信した後、`parse`関数が呼び出されます。

このSpiderを使い始めるには、2つのことをする必要があります：

- start_urls`をスクレイピングしたいURL <https://www.chocolate.co.uk/collections/all>に変更する。
- パースコードを `parse` 関数に挿入する。

### Step 4 - Update Start Urls

```python
# chocolatespider.py

import scrapy

class ChocolatespiderSpider(scrapy.Spider):
    name = 'chocolatespider'
    allowed_domains = ['chocolate.co.uk']
    start_urls = ['https://chocolate.co.uk/collections/all']

    def parse(self, response):
        pass
```

次に、ページから必要なデータを解析するためのCSSセレクタを作成する必要があります。

これを行うには、Scrapy Shellを使用します。

### Step 5 - Scrapy Shell: Finding Our CSS Selectors

HTMLページからデータを抽出するには、[XPath](https://www.w3schools.com/xml/xpath_intro.asp)または[CSSセレクタ](https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Selectors)を使用して、Scrapyにページのどこにデータがあるのかを伝える必要があります。XPathとCSSセレクタは、ScrapyがDOMツリーをナビゲートし、必要なデータの場所を見つけるための小さなマップのようなものです。このガイドでは、ページからデータを解析するためにCSSセレクタを使用するつもりです。そして、これらのCSSセレクタを作成するために、私たちは[Scrapy Shell](https://docs.scrapy.org/en/latest/topics/shell.html)を使用する予定です。

Scrapyの素晴らしい機能の1つは、あなたがすぐにあなたのXPathとCSSセレクタをテストし、デバッグすることができ、組み込みのシェルが付属していることです。XPathやCSSのセレクタが正しいかどうかを確認するためにスクレイパーを実行する代わりに、ターミナルに直接入力して結果を確認することができます。

```bash
# Scrapyシェルを開くには、次のコマンドを使用します：
(venv) chocolatescraper (😁 :main *) :$ scrapy shell

2023-05-29 20:06:24 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: chocolatescraper)
2023-05-29 20:06:24 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (main, Apr 24 2023, 17:34:58) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform macOS-13.3.1-x86_64-i386-64bit
2023-05-29 20:06:24 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'chocolatescraper',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'FEED_EXPORT_ENCODING': 'utf-8',
 'LOGSTATS_INTERVAL': 0,
 'NEWSPIDER_MODULE': 'chocolatescraper.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['chocolatescraper.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2023-05-29 20:06:24 [asyncio] DEBUG: Using selector: KqueueSelector
2023-05-29 20:06:24 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-05-29 20:06:24 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2023-05-29 20:06:24 [scrapy.extensions.telnet] INFO: Telnet Password: f3038c0cc138d466
2023-05-29 20:06:24 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage']
2023-05-29 20:06:24 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2023-05-29 20:06:24 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2023-05-29 20:06:24 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2023-05-29 20:06:24 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2023-05-29 20:06:24 [asyncio] DEBUG: Using selector: KqueueSelector
[s] Available Scrapy objects:
[s]   scrapy     scrapy module (contains scrapy.Request, scrapy.Selector, etc)
[s]   crawler    <scrapy.crawler.Crawler object at 0x10dcf76a0>
[s]   item       {}
[s]   settings   <scrapy.settings.Settings object at 0x10dcf6ce0>
[s] Useful shortcuts:
[s]   fetch(url[, redirect=True]) Fetch URL and update local objects (by default, redirects are followed)
[s]   fetch(req)                  Fetch a scrapy.Request and update local objects 
[s]   shelp()           Shell help (print this help)
[s]   view(response)    View response in a browser
2023-05-29 20:06:25 [asyncio] DEBUG: Using selector: KqueueSelector
In [1]: 
```

```bash
# scrapy shellを抜ける
In [1]: quit
```

#### Fetch The Page

CSSセレクタを作成するために、次のページでテストします：

<https://www.chocolate.co.uk/collections/all>

```bash
(venv) chocolatescraper (😁 :main) :$ scrapy shell

2023-05-30 20:15:36 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: chocolatescraper)
2023-05-30 20:15:36 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (main, Apr 24 2023, 17:34:58) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform macOS-13.3.1-x86_64-i386-64bit
2023-05-30 20:15:36 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'chocolatescraper',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'FEED_EXPORT_ENCODING': 'utf-8',
 'LOGSTATS_INTERVAL': 0,
 'NEWSPIDER_MODULE': 'chocolatescraper.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['chocolatescraper.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2023-05-30 20:15:36 [asyncio] DEBUG: Using selector: KqueueSelector
2023-05-30 20:15:36 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-05-30 20:15:36 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2023-05-30 20:15:36 [scrapy.extensions.telnet] INFO: Telnet Password: a9bea5e1b6db9314
2023-05-30 20:15:36 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage']
2023-05-30 20:15:36 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2023-05-30 20:15:36 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2023-05-30 20:15:36 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2023-05-30 20:15:36 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2023-05-30 20:15:37 [asyncio] DEBUG: Using selector: KqueueSelector
[s] Available Scrapy objects:
[s]   scrapy     scrapy module (contains scrapy.Request, scrapy.Selector, etc)
[s]   crawler    <scrapy.crawler.Crawler object at 0x10aa8a2f0>
[s]   item       {}
[s]   settings   <scrapy.settings.Settings object at 0x10aa8b010>
[s] Useful shortcuts:
[s]   fetch(url[, redirect=True]) Fetch URL and update local objects (by default, redirects are followed)
[s]   fetch(req)                  Fetch a scrapy.Request and update local objects 
[s]   shelp()           Shell help (print this help)
[s]   view(response)    View response in a browser
2023-05-30 20:15:37 [asyncio] DEBUG: Using selector: KqueueSelector
```

```bash
# チョコレートサイトのメイン商品ページを取得
In [1]: fetch('https://www.chocolate.co.uk/collections/all')
2023-05-30 20:16:32 [scrapy.core.engine] INFO: Spider opened
2023-05-30 20:16:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/robots.txt> (referer: None)
2023-05-30 20:16:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/collections/all> (referer: None)
```

```bash
# Scrapyシェルは自動的にHTMLレスポンスをresponse変数に保存している
In [2]: response
Out[2]: <200 https://www.chocolate.co.uk/collections/all>
```

#### 商品のCSSセレクタを探す

```bash
# `product-item`クラスを使って商品情報を抽出する
In [3]: response.css('product-item')
Out[3]: 
[<Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item pro...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>,
 <Selector query='descendant-or-self::product-item' data='<product-item class="product-item " r...'>]
```

#### Get First Product

```bash
# get()を使用して、最初の商品を取得する
In [4]: response.css('product-item').get()
Out[4]: '<product-item class="product-item " reveal><div class="product-item__image-wrapper product-item__image-wrapper--multiple"><div class="product-item__label-list label-list"><span class="label label--custom">New</span><span class="label label--highlight">Save £5.95</span></div><a href="/products/100-dark-hot-chocolate-flakes" class="product-item__aspect-ratio aspect-ratio aspect-ratio--square" style="padding-bottom: 100.0%; --aspect-ratio: 1.0">\n      <img class="product-item__primary-image" loading="lazy" data-media-id="20964759797803" sizes="(max-width: 740px) calc(50vw - 24px), calc((min(100vw - 80px, 1520px) - 305px) / 4 - 18px)" height="1200" width="1200" alt="100% Dark Hot Chocolate Flakes" src="//cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_1200x.jpg?v=1638088899" srcset="//cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_200x.jpg?v=1638088899 200w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_300x.jpg?v=1638088899 300w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_400x.jpg?v=1638088899 400w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_500x.jpg?v=1638088899 500w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_600x.jpg?v=1638088899 600w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_700x.jpg?v=1638088899 700w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_800x.jpg?v=1638088899 800w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_900x.jpg?v=1638088899 900w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_1000x.jpg?v=1638088899 1000w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_1100x.jpg?v=1638088899 1100w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-dark-hot-chocolate-flakes_1200x.jpg?v=1638088899 1200w"><img class="product-item__secondary-image" loading="lazy" sizes="(max-width: 740px) calc(50vw - 24px), calc((min(100vw - 80px, 1520px) - 305px) / 4 - 18px)" height="1200" width="1200" alt="100% Dark Hot Chocolate Flakes" src="//cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_1200x.jpg?v=1638088899" srcset="//cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_200x.jpg?v=1638088899 200w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_300x.jpg?v=1638088899 300w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_400x.jpg?v=1638088899 400w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_500x.jpg?v=1638088899 500w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_600x.jpg?v=1638088899 600w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_700x.jpg?v=1638088899 700w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_800x.jpg?v=1638088899 800w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_900x.jpg?v=1638088899 900w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_1000x.jpg?v=1638088899 1000w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_1100x.jpg?v=1638088899 1100w, //cdn.shopify.com/s/files/1/1991/9591/products/100-percent-hot-chocolate_1200x.jpg?v=1638088899 1200w"></a><form method="post" action="/cart/add" id="product_form_template--14852576477227__main__6651573895211_0" accept-charset="UTF-8" class="product-item__quick-form" enctype="multipart/form-data" is="product-form"><input type="hidden" name="form_type" value="product"><input type="hidden" name="utf8" value="✓"><input type="hidden" name="quantity" value="1">\n          <input type="hidden" name="id" value="39589492621355">\n          <button is="loader-button" type="submit" class="button button--outline button--text button--full  hidden-touch">+ Add to cart</button>\n          <button type="submit" class="product-item__quick-buy-button hidden-no-touch">\n            <span class="visually-hidden">+ Add to cart</span><svg focusable="false" width="24" height="24" class="icon icon--quick-buy-shopping-cart   " fill="none" viewbox="0 0 24 24">\n        <path d="M1 3H5L6 13H18L20 6H18" stroke="currentColor" stroke-width="1.7"></path>\n        <circle cx="7" cy="19" r="2" stroke="currentColor" stroke-width="1.7"></circle>\n        <circle cx="17" cy="19" r="2" stroke="currentColor" stroke-width="1.7"></circle>\n        <path d="M12 2V10M8 6H16" stroke="currentColor" stroke-width="1.7"></path>\n      </svg></button></form></div>\n\n  <div class="product-item__info  ">\n    <div class="product-item-meta"><a href="/products/100-dark-hot-chocolate-flakes" class="product-item-meta__title">100% Dark Hot Chocolate Flakes</a>\n\n      <div class="product-item-meta__price-list-container">\n        <div class="price-list price-list--centered"><span class="price price--highlight">\n              <span class="visually-hidden">Sale price</span>£4.00</span>\n\n            <span class="price price--compare">\n              <span class="visually-hidden">Regular price</span>£9.95</span></div>\n      </div></div></div>\n</product-item>'
```

#### 全商品を見る

```bash
In [5]: products = response.css('product-item')

In [6]: len(products)
Out[6]: 24
```

#### エクストラクト商品詳細

- 商品リストから各商品の名前、価格、URLを抽出

```bash
# リストの最初の要素で CSS セレクターをテストする
# 商品名
In [7]: product = products[0]

In [8]: product.css('a.product-item-meta__title::text').get()
Out[8]: '100% Dark Hot Chocolate Flakes'
```

```bash
# 商品価格の抽出
In [6]: product.css('span.price').get()
Out[6]: '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price</span>▒8.50</span>'
```

```bash
# 余分なタグの削除
In [14]: product.css('span.price').get().replace('<span class="price price--highlight">\n              <span class="visually-h
    ...: idden">Sale price</span>','').replace('</span>','')
Out[14]: '£4.00'
```

```bash
# 商品URLの抽出
In [12]: product.css('div.product-item-meta a').attrib['href']
Out[12]: '/products/100-dark-hot-chocolate-flakes'
```

#### Updated Spider

```python
# chocolatespider.py
import scrapy

class ChocolatespiderSpider(scrapy.Spider):

    # spiderの名前
    name = 'chocolatespider'

    # the url of the first page that we will start scraping
    # スクレイピングを開始する最初のページのURL
    start_urls = ['https://www.chocolate.co.uk/collections/all']

    def parse(self, response):

        # here we are looping through the products and extracting the name, price & url
        # 商品をループして、名前、価格、URLを抽出
        products = response.css('product-item')
        for product in products:
            # here we put the data returned into the format we want to output for our csv or json file
            # 返されたデータをcsvファイルやjsonファイルに出力したい形式に変換
            yield{
                'name' : product.css('a.product-item-meta__title::text').get(),
                'price' : product.css('span.price').get().replace('<span class="price">\n              <span class="visually-hidden">Sale price</span>','').replace('</span>',''),
                'url' : product.css('div.product-item-meta a').attrib['href'],
            }
```

### Step 6 - Spiderを走らせる

```bash
# Spiderの実行
(venv) chocolatescraper (😁 :main *) :$ scrapy crawl chocolatespider

2023-06-02 21:30:25 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: chocolatescraper)
2023-06-02 21:30:25 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (main, Apr 24 2023, 17:34:58) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform macOS-13.3.1-x86_64-i386-64bit
2023-06-02 21:30:25 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'chocolatescraper',
 'FEED_EXPORT_ENCODING': 'utf-8',
 'NEWSPIDER_MODULE': 'chocolatescraper.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['chocolatescraper.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2023-06-02 21:30:25 [asyncio] DEBUG: Using selector: KqueueSelector
2023-06-02 21:30:25 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-06-02 21:30:25 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2023-06-02 21:30:25 [scrapy.extensions.telnet] INFO: Telnet Password: c06af8d80b09ffe1
2023-06-02 21:30:25 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2023-06-02 21:30:26 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2023-06-02 21:30:26 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2023-06-02 21:30:26 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2023-06-02 21:30:26 [scrapy.core.engine] INFO: Spider opened
2023-06-02 21:30:26 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2023-06-02 21:30:26 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2023-06-02 21:30:27 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/robots.txt> (referer: None)
2023-06-02 21:30:28 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/collections/all> (referer: None)
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '100% Dark Hot Chocolate Flakes', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£4.00', 'url': '/products/100-dark-hot-chocolate-flakes'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '2.5kg Bulk 41% Milk Hot Chocolate Drops', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£35.00', 'url': '/products/2-5kg-bulk-of-our-41-milk-hot-chocolate-drops'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '2.5kg Bulk 61% Dark Hot Chocolate Drops', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£35.00', 'url': '/products/2-5kg-of-our-best-selling-61-dark-hot-chocolate-drops'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '41% Milk Hot Chocolate Drops', 'price': '£8.75', 'url': '/products/41-colombian-milk-hot-chocolate-drops'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '61% Dark Hot Chocolate Drops', 'price': '£8.75', 'url': '/products/62-dark-hot-chocolate'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '70% Dark Hot Chocolate Flakes', 'price': '£9.95', 'url': '/products/70-dark-hot-chocolate-flakes'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Almost Perfect', 'price': 'From £1.50\n', 'url': '/products/almost-perfect'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Assorted Chocolate Malt Balls', 'price': '£9.00', 'url': '/products/assorted-chocolate-malt-balls'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Caramel', 'price': '£5.00', 'url': '/products/blonde-caramel-chocolate-bar'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Chocolate Honeycomb', 'price': '£9.00', 'url': '/products/blonde-chocolate-honeycomb'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Chocolate Honeycomb - Bag', 'price': '£8.50', 'url': '/products/blonde-chocolate-sea-salt-honeycomb'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Chocolate Malt Balls', 'price': '£9.00', 'url': '/products/blonde-chocolate-malt-balls'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Chocolate Truffles', 'price': '£19.95', 'url': '/products/blonde-chocolate-truffles'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Hot Chocolate Flakes', 'price': '£9.95', 'url': '/products/blonde-hot-chocolate-flakes'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Bulk 41% Milk Hot Chocolate Drops 750 grams', 'price': '£17.50', 'url': '/products/bulk-41-milk-hot-chocolate-drops-750-grams'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Bulk 61% Dark Hot Chocolate Drops 750 grams', 'price': '£17.50', 'url': '/products/750-gram-bulk-61-dark-hot-chocolate-drops'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Caramelised Milk', 'price': '£5.00', 'url': '/products/caramelised-milk-chocolate-bar'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Chocolate Caramelised Pecan Nuts', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£4.50', 'url': '/products/chocolate-caramelised-pecan-nuts'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Chocolate Celebration Hamper', 'price': '£55.00', 'url': '/products/celebration-hamper'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Cinnamon Toast', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£3.00', 'url': '/products/cinnamon-toast-chocolate-bar'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Collection of 4 of our Best Selling Chocolate Malt Balls', 'price': '£30.00', 'url': '/products/collection-of-our-best-selling-chocolate-malt-balls'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Colombia 61%', 'price': '£5.00', 'url': '/products/colombian-dark-chocolate-bar'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Colombian 41%', 'price': '£5.00', 'url': '/products/ecuadorian-41-milk-chocolate-bar'}
2023-06-02 21:30:28 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Crunchy Biscuit', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£3.00', 'url': '/products/crunchy-biscuit-blonde-chocolate-bar'}
2023-06-02 21:30:28 [scrapy.core.engine] INFO: Closing spider (finished)
2023-06-02 21:30:28 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 960,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 47672,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 2,
 'elapsed_time_seconds': 2.381835,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 6, 2, 12, 30, 28, 560781),
 'httpcompression/response_bytes': 237468,
 'httpcompression/response_count': 2,
 'item_scraped_count': 24,
 'log_count/DEBUG': 29,
 'log_count/INFO': 10,
 'memusage/max': 57081856,
 'memusage/startup': 57081856,
 'response_received_count': 2,
 'robotstxt/request_count': 1,
 'robotstxt/response_count': 1,
 'robotstxt/response_status_count/200': 1,
 'scheduler/dequeued': 1,
 'scheduler/dequeued/memory': 1,
 'scheduler/enqueued': 1,
 'scheduler/enqueued/memory': 1,
 'start_time': datetime.datetime(2023, 6, 2, 12, 30, 26, 178946)}
2023-06-02 21:30:28 [scrapy.core.engine] INFO: Spider closed (finished)
```

```bash
# JSONファイルに保存
scrapy crawl chocolatespider -O myscrapeddata.json
```

```bash
# CSVファイルに保存
scrapy crawl chocolatespider -O myscrapeddata.csv
```

### Step 6 - "Next Page"に移動する

```bash
(venv) chocolatescraper (😁 :main *) :$ scrapy shell
2023-06-02 21:37:31 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: chocolatescraper)
2023-06-02 21:37:31 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (main, Apr 24 2023, 17:34:58) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform macOS-13.3.1-x86_64-i386-64bit
2023-06-02 21:37:31 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'chocolatescraper',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'FEED_EXPORT_ENCODING': 'utf-8',
 'LOGSTATS_INTERVAL': 0,
 'NEWSPIDER_MODULE': 'chocolatescraper.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['chocolatescraper.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2023-06-02 21:37:31 [asyncio] DEBUG: Using selector: KqueueSelector
2023-06-02 21:37:31 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-06-02 21:37:31 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2023-06-02 21:37:31 [scrapy.extensions.telnet] INFO: Telnet Password: ccaffdca7bb66c37
2023-06-02 21:37:31 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage']
2023-06-02 21:37:31 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2023-06-02 21:37:31 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2023-06-02 21:37:31 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2023-06-02 21:37:31 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2023-06-02 21:37:32 [asyncio] DEBUG: Using selector: KqueueSelector
[s] Available Scrapy objects:
[s]   scrapy     scrapy module (contains scrapy.Request, scrapy.Selector, etc)
[s]   crawler    <scrapy.crawler.Crawler object at 0x10e53a110>
[s]   item       {}
[s]   settings   <scrapy.settings.Settings object at 0x10e53b010>
[s] Useful shortcuts:
[s]   fetch(url[, redirect=True]) Fetch URL and update local objects (by default, redirects are followed)
[s]   fetch(req)                  Fetch a scrapy.Request and update local objects 
[s]   shelp()           Shell help (print this help)
[s]   view(response)    View response in a browser
2023-06-02 21:37:32 [asyncio] DEBUG: Using selector: KqueueSelector
```

```bash
In [1]: fetch('https://www.chocolate.co.uk/collections/all')
2023-06-02 21:37:39 [scrapy.core.engine] INFO: Spider opened
2023-06-02 21:37:40 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/robots.txt> (referer: None)
2023-06-02 21:37:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/collections/all> (referer: None)
```

```bash
In [2]: response.css('[rel="next"] ::attr(href)').get()
Out[2]: '/collections/all?page=2'
```

```bash
In [3]: response.css('[rel="next"] ::attr(href)').get()
Out[3]: '/collections/all?page=2'
```

```bash
(venv) chocolatescraper (😁 :main *) :$ scrapy crawl chocolatespider

2023-06-02 21:44:29 [scrapy.utils.log] INFO: Scrapy 2.9.0 started (bot: chocolatescraper)
2023-06-02 21:44:29 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.1, Twisted 22.10.0, Python 3.10.11 (main, Apr 24 2023, 17:34:58) [Clang 14.0.3 (clang-1403.0.22.14.1)], pyOpenSSL 23.1.1 (OpenSSL 3.1.0 14 Mar 2023), cryptography 40.0.2, Platform macOS-13.3.1-x86_64-i386-64bit
2023-06-02 21:44:29 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'chocolatescraper',
 'FEED_EXPORT_ENCODING': 'utf-8',
 'NEWSPIDER_MODULE': 'chocolatescraper.spiders',
 'REQUEST_FINGERPRINTER_IMPLEMENTATION': '2.7',
 'ROBOTSTXT_OBEY': True,
 'SPIDER_MODULES': ['chocolatescraper.spiders'],
 'TWISTED_REACTOR': 'twisted.internet.asyncioreactor.AsyncioSelectorReactor'}
2023-06-02 21:44:29 [asyncio] DEBUG: Using selector: KqueueSelector
2023-06-02 21:44:29 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.asyncioreactor.AsyncioSelectorReactor
2023-06-02 21:44:29 [scrapy.utils.log] DEBUG: Using asyncio event loop: asyncio.unix_events._UnixSelectorEventLoop
2023-06-02 21:44:29 [scrapy.extensions.telnet] INFO: Telnet Password: f66b2221a91c9d83
2023-06-02 21:44:29 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2023-06-02 21:44:29 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.robotstxt.RobotsTxtMiddleware',
 'scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'scrapy.downloadermiddlewares.redirect.RedirectMiddleware',
 'scrapy.downloadermiddlewares.cookies.CookiesMiddleware',
 'scrapy.downloadermiddlewares.httpproxy.HttpProxyMiddleware',
 'scrapy.downloadermiddlewares.stats.DownloaderStats']
2023-06-02 21:44:29 [scrapy.middleware] INFO: Enabled spider middlewares:
['scrapy.spidermiddlewares.httperror.HttpErrorMiddleware',
 'scrapy.spidermiddlewares.offsite.OffsiteMiddleware',
 'scrapy.spidermiddlewares.referer.RefererMiddleware',
 'scrapy.spidermiddlewares.urllength.UrlLengthMiddleware',
 'scrapy.spidermiddlewares.depth.DepthMiddleware']
2023-06-02 21:44:29 [scrapy.middleware] INFO: Enabled item pipelines:
[]
2023-06-02 21:44:29 [scrapy.core.engine] INFO: Spider opened
2023-06-02 21:44:29 [scrapy.extensions.logstats] INFO: Crawled 0 pages (at 0 pages/min), scraped 0 items (at 0 items/min)
2023-06-02 21:44:29 [scrapy.extensions.telnet] INFO: Telnet console listening on 127.0.0.1:6023
2023-06-02 21:44:30 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/robots.txt> (referer: None)
2023-06-02 21:44:31 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/collections/all> (referer: None)
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '100% Dark Hot Chocolate Flakes', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£4.00', 'url': '/products/100-dark-hot-chocolate-flakes'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '2.5kg Bulk 41% Milk Hot Chocolate Drops', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£35.00', 'url': '/products/2-5kg-bulk-of-our-41-milk-hot-chocolate-drops'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '2.5kg Bulk 61% Dark Hot Chocolate Drops', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£35.00', 'url': '/products/2-5kg-of-our-best-selling-61-dark-hot-chocolate-drops'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '41% Milk Hot Chocolate Drops', 'price': '£8.75', 'url': '/products/41-colombian-milk-hot-chocolate-drops'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '61% Dark Hot Chocolate Drops', 'price': '£8.75', 'url': '/products/62-dark-hot-chocolate'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': '70% Dark Hot Chocolate Flakes', 'price': '£9.95', 'url': '/products/70-dark-hot-chocolate-flakes'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Almost Perfect', 'price': 'From £1.50\n', 'url': '/products/almost-perfect'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Assorted Chocolate Malt Balls', 'price': '£9.00', 'url': '/products/assorted-chocolate-malt-balls'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Caramel', 'price': '£5.00', 'url': '/products/blonde-caramel-chocolate-bar'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Chocolate Honeycomb', 'price': '£9.00', 'url': '/products/blonde-chocolate-honeycomb'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Chocolate Honeycomb - Bag', 'price': '£8.50', 'url': '/products/blonde-chocolate-sea-salt-honeycomb'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Chocolate Malt Balls', 'price': '£9.00', 'url': '/products/blonde-chocolate-malt-balls'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Chocolate Truffles', 'price': '£19.95', 'url': '/products/blonde-chocolate-truffles'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Blonde Hot Chocolate Flakes', 'price': '£9.95', 'url': '/products/blonde-hot-chocolate-flakes'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Bulk 41% Milk Hot Chocolate Drops 750 grams', 'price': '£17.50', 'url': '/products/bulk-41-milk-hot-chocolate-drops-750-grams'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Bulk 61% Dark Hot Chocolate Drops 750 grams', 'price': '£17.50', 'url': '/products/750-gram-bulk-61-dark-hot-chocolate-drops'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Caramelised Milk', 'price': '£5.00', 'url': '/products/caramelised-milk-chocolate-bar'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Chocolate Caramelised Pecan Nuts', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£4.50', 'url': '/products/chocolate-caramelised-pecan-nuts'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Chocolate Celebration Hamper', 'price': '£55.00', 'url': '/products/celebration-hamper'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Cinnamon Toast', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£3.00', 'url': '/products/cinnamon-toast-chocolate-bar'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Collection of 4 of our Best Selling Chocolate Malt Balls', 'price': '£30.00', 'url': '/products/collection-of-our-best-selling-chocolate-malt-balls'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Colombia 61%', 'price': '£5.00', 'url': '/products/colombian-dark-chocolate-bar'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Colombian 41%', 'price': '£5.00', 'url': '/products/ecuadorian-41-milk-chocolate-bar'}
2023-06-02 21:44:31 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all>
{'name': 'Crunchy Biscuit', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£3.00', 'url': '/products/crunchy-biscuit-blonde-chocolate-bar'}
2023-06-02 21:44:32 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/collections/all?page=2> (referer: https://www.chocolate.co.uk/collections/all)
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Dark Chocolate Ginger', 'price': '£8.95', 'url': '/products/dark-chocolate-ginger'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Dark Chocolate Honeycomb', 'price': '£9.00', 'url': '/products/dark-chocolate-honeycomb-1'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Dark Chocolate Honeycomb - Bag', 'price': '£8.50', 'url': '/products/dark-chocolate-honeycomb'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Dark Chocolate Malt Balls', 'price': '£9.00', 'url': '/products/dark-chocolate-malt-balls'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Dark Chocolate Orange Peel', 'price': '£8.95', 'url': '/products/dark-chocolate-orange-peel'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Dark Chocolate Truffles', 'price': '£19.95', 'url': '/products/dark-chocolate-truffles'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Flat White Coffee', 'price': '£5.00', 'url': '/products/flat-white-coffee'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Gift Voucher', 'price': 'From £10.00\n', 'url': '/products/gift-voucher'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Ginger Biscuit', 'price': '£5.00', 'url': '/products/ginger-biscuit-chocolate-bar'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Honeycomb Crunch', 'price': '£5.00', 'url': '/products/honeycomb-crunch'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Hot Chocolate Shaker', 'price': 'From £25.00\n', 'url': '/products/hot-chocolate-shaker-new'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'June Box of the Month', 'price': '<span class="price">\n                <span class="visually-hidden">Sale priceFrom £30.00', 'url': '/products/box-of-the-month-subscription-1'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Luxury Chocolate Hamper', 'price': '£85.00', 'url': '/products/luxury-chocolate-hamper'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Luxury Chocolate Hamper with bottle of Champagne', 'price': '£100.00', 'url': '/products/copy-of-luxury-chocolate-hamper'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Luxury Hot Chocolate Hamper', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£50.00', 'url': '/products/luxury-hot-chocolate-hamper'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Marc De Champagne Truffles', 'price': '£19.95', 'url': '/products/marc-de-champagne-truffles'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Marmalade', 'price': '£5.00', 'url': '/products/marmalade-orange-chocolate-bar'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Milk Chocolate Honeycomb', 'price': '£9.00', 'url': '/products/milk-chocolate-honeycomb-2'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Milk Chocolate Honeycomb - Bag', 'price': '£8.50', 'url': '/products/milk-chocolate-honeycomb'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Milk Chocolate Malt Balls', 'price': '£9.00', 'url': '/products/milk-chocolate-malt-balls'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Milk Chocolate Truffles', 'price': '£19.95', 'url': '/products/milk-chocolate-truffles'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Mixed Chocolate Mini Eggs', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£12.00', 'url': '/products/mixed-milk-chocolate-mini-eggs'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': "Mother's Day Mystery Hamper", 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£50.00', 'url': '/products/mystery-box'}
2023-06-02 21:44:32 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=2>
{'name': 'Mystery Box', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£30.00', 'url': '/products/mothers-day-mystery-hamper'}
2023-06-02 21:44:33 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/collections/all?page=3> (referer: https://www.chocolate.co.uk/collections/all?page=2)
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Oat M!lk Hot Chocolate Flakes', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£4.00', 'url': '/products/oat-m-lk-hot-chocolate-flakes'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Orange Dark Hot Chocolate Flakes', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£4.00', 'url': '/products/orange-dark-hot-chocolate-flakes'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Orange Malt Balls', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£5.00', 'url': '/products/orange-malt-balls'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Peppermint Malt Balls', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£5.00', 'url': '/products/peppermint-malt-balls'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Peppermint Swirl', 'price': '£5.00', 'url': '/products/peppermint-swirl-chocolate-bar'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Pistachio & Cranberry', 'price': '£5.00', 'url': '/products/pistachio-cranberry-chocolate-bar'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Praline Quail Eggs', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£4.50', 'url': '/products/praline-quail-eggs'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Pretzel Caramel', 'price': '£5.00', 'url': '/products/pretzel-caramel-chocolate-bar'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Roasted Almonds', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£4.50', 'url': '/products/cocoa-dusted-almonds'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Salt Caramel Malt Balls', 'price': '£9.00', 'url': '/products/salt-caramel-malt-balls'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Salt Caramelised Hazelnuts', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£4.50', 'url': '/products/salt-caramelised-hazelnuts'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Salted Caramel', 'price': '£5.00', 'url': '/products/blonde-chocolate-sea-salt'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Salted Caramel Florentines', 'price': '£11.50', 'url': '/products/salted-caramel-florentines'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Salted Caramel Quail Eggs', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£5.00', 'url': '/products/salted-caramel-quail-eggs'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Salted Caramels Blonde Chocolate', 'price': '£19.95', 'url': '/products/salted-caramels-blonde-chocolate'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Salted Caramels Dark Chocolate', 'price': '£19.95', 'url': '/products/salt-caramels-dark-chocolate'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Salted Caramels Milk Chocolate', 'price': '£19.95', 'url': '/products/salt-caramels-milk-chocolate'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Valrhona Araguani - 72%', 'price': '£77.50', 'url': '/products/valrhona-araguani-72'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Valrhona Bahibe 46%', 'price': '£75.00', 'url': '/products/valrhona-bahibe'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Valrhona Biskelia 34%', 'price': '£75.00', 'url': '/products/valrhona-biskelia-34'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Valrhona Caraibe 66%', 'price': '£75.00', 'url': '/products/valrhona-caraibe-66'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Valrhona Caramelia 36%', 'price': '£75.00', 'url': '/products/valrhona-caramelia-36'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Valrhona Dulcey 32%', 'price': '£77.50', 'url': '/products/valrhona-dulcey-32'}
2023-06-02 21:44:33 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=3>
{'name': 'Valrhona Guanaja 70%', 'price': '£75.00', 'url': '/products/valrhona-guanaja-70'}
2023-06-02 21:44:34 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.chocolate.co.uk/collections/all?page=4> (referer: https://www.chocolate.co.uk/collections/all?page=3)
2023-06-02 21:44:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=4>
{'name': 'Valrhona Jivara 40%', 'price': '£75.00', 'url': '/products/valrhona-jivara-40'}
2023-06-02 21:44:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=4>
{'name': 'Valrhona Manjari 64%', 'price': '£75.00', 'url': '/products/valrhona-manjari-64'}
2023-06-02 21:44:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=4>
{'name': 'Valrhona Opalys 33%', 'price': '£75.00', 'url': '/products/valrhona-opalys-33'}
2023-06-02 21:44:34 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.chocolate.co.uk/collections/all?page=4>
{'name': 'Velvet White', 'price': '<span class="price price--highlight">\n              <span class="visually-hidden">Sale price£2.50', 'url': '/products/velvet-white-chocolate-bar'}
2023-06-02 21:44:34 [scrapy.core.engine] INFO: Closing spider (finished)
2023-06-02 21:44:34 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 3509,
 'downloader/request_count': 5,
 'downloader/request_method_count/GET': 5,
 'downloader/response_bytes': 170058,
 'downloader/response_count': 5,
 'downloader/response_status_count/200': 5,
 'elapsed_time_seconds': 4.463839,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2023, 6, 2, 12, 44, 34, 274547),
 'httpcompression/response_bytes': 813970,
 'httpcompression/response_count': 5,
 'item_scraped_count': 76,
 'log_count/DEBUG': 84,
 'log_count/INFO': 10,
 'memusage/max': 57143296,
 'memusage/startup': 57143296,
 'request_depth_max': 3,
 'response_received_count': 5,
 'robotstxt/request_count': 1,
 'robotstxt/response_count': 1,
 'robotstxt/response_status_count/200': 1,
 'scheduler/dequeued': 4,
 'scheduler/dequeued/memory': 4,
 'scheduler/enqueued': 4,
 'scheduler/enqueued/memory': 4,
 'start_time': datetime.datetime(2023, 6, 2, 12, 44, 29, 810708)}
2023-06-02 21:44:34 [scrapy.core.engine] INFO: Spider closed (finished)
```